# 线程

**线程**（**thread**）是计算机并行编程的一种方法，可以允许在一个**进程**（**process**）内同时完成多个任务，而同时在一个进程内不同的线程支架可以共享数据。

比如，在网络爬虫的设计中，一般的模式是：
```
获取数据 ->  解析数据 -> 存储数据
```
如果按照这样的线性的操作，整个过程的效率决定于最慢的一个环节。比如，通常来讲解析和存储数据是非常快的，而获取数据的步骤由于涉及到网络IO，速度比较慢，一个想法是我们可以将三者解耦，即分成两个不同的部分：
```
网络IO部分：

获取数据 ~> 暂存区域
```
以及：
```
本地IO部分：

暂存数据 ~> 解析数据 -> 存储数据
```
其中网络IO部分和本地IO部分**同时**运行，其交流通过一个暂存区域进行。

按照这个方法

* 网络IO部分在获取数据以后可以立即进行下一次的数据获取，而不需要等待解析数据和存储数据的完成；甚至这个部分可以复制两份、三份，即有三个不同的线程分别都在进行获取数据工作
* 本地IO部分不断的查看暂存区域，如果有信息就将其解析，并进行本地存储。

这样，通过将任务解耦，在一个程序内部有几个不同的线程进行分工合作，就可以极大的提高工作效率。

然而，**多个线程共同访问一个存储区域是一个危险的行为**！

比如，一个简单的例子，如果有多个线程同时向暂存区域写入数据，不同的进程之间有可能会打架，相互覆盖掉彼此的数据，此时就会引起明显的冲突。

解决这一问题的简单方案是加一个“锁”：当一个线程写入一个变量或者区域时，其他线程不得对该变量或者区域进行操作。

而值得注意的是，为了防止以上情况的发生，Python使用了**全局解释器锁**（**GIL**），这个锁可以保证同时只有一个线程在运行。

所以Python中的多线程实际上是假的多线程：自始至终在同一时间只有一个线程在工作，只不过Python在不同时间切换到了不同线程进行工作，看起来好像是多线程在工作一样。这有点像我们平时微信聊天：在每个时间点，我们只能跟一个人聊天，但是这不影响我们这一分钟跟A聊天，下一分钟跟B聊天，再下一分钟再返回来跟A聊天，这样整个看起来我好像是“同时”在跟两个人聊天一样。

正因为有GIL，所以Python中的多线程的效率是非常低的，所以在科学计算等高性能计算领域，Python的多线程对于提高计算速度并没有任何帮助（NumPy等工具箱的并行计算依赖于更加底层的BLAS等，本质上是一些C、Fortran写出来的计算包，所以实际运算的时候并不是使用的Python的多线程计算，而是使用的C、Fortran的并行计算，并没有GIL的限制，效率非常高）。然而在一些IO密集型的领域，比如网络爬虫，性能的瓶颈并不是计算，而是IO，此时Python线程的使用可以大大提高效率。

接下来我们使用几个简单的例子介绍Python中多线程编程的使用方法。

在Python中，可以通过threading模块引入线程，在threading中有一个Threading类，我们所要做的就是继承Threading这个类，然后定义初始化方法以及运行方法，比如

In [1]:
import threading
import time

class simple_thread(threading.Thread):
    def __init__(self, delta_time, name):
        threading.Thread.__init__(self)
        self.name=name
        self.delta_time=delta_time
    def run(self):
        t=0
        while t<=10:
            print(f"In thread {self.name}, t={t}")
            time.sleep(self.delta_time)
            t+=1

以上仅仅定义了一个进程，还没有开始运行，为了运行，我们可以创建以上对象的两个实例：

In [2]:
thread1=simple_thread(1,'A')
thread2=simple_thread(0.5,'B')

其中thread1每隔1秒打印一次，而thread2每隔0.5秒打印一次。

然后使用start()方法启动：

In [3]:
thread1.start()
thread2.start()
thread1.join()
thread2.join()
print("所有线程结束")

In thread A, t=0
In thread B, t=0
In thread B, t=1
In thread A, t=1
In thread B, t=2
In thread B, t=3
In thread A, t=2
In thread B, t=4
In thread B, t=5
In thread A, t=3
In thread B, t=6
In thread B, t=7
In thread A, t=4
In thread B, t=8
In thread B, t=9
In thread A, t=5
In thread B, t=10
In thread A, t=6
In thread A, t=7
In thread A, t=8
In thread A, t=9
In thread A, t=10
所有线程结束


其中join()方法意思是让调用该线程的主线程等待该线程运行结束。

此外，还可以使用is_alive()方法判断该线程是否仍在活动，比如可以再写一个线程对其进行监听：

In [4]:
thread1=simple_thread(1,'A')
thread2=simple_thread(0.5,'B')
Threads=[thread1, thread2]

class monitor_thread(threading.Thread):
    def __init__(self, threads):
        threading.Thread.__init__(self)
        self.threads={}
        for i in range(len(threads)):
            self.threads[i]={}
            self.threads[i]["thread"]=threads[i]
            self.threads[i]["alive"]=True
    def run(self):
        while True:
            for t in self.threads:
                if self.threads[t]["alive"] and not self.threads[t]["thread"].is_alive():
                    self.threads[t]["alive"]=False
                    print("Thread %s is down." % self.threads[t]["thread"].getName())
            all_done=True
            for t in self.threads:
                if self.threads[t]["alive"]:
                    all_done=False
                    break
            if all_done:
                break
            time.sleep(0.9)
thread3=monitor_thread(Threads)

for t in Threads:
    t.start()
thread3.start()
for t in Threads:
    t.join()
thread3.join()

In thread A, t=0
In thread B, t=0
In thread B, t=1
In thread A, t=1
In thread B, t=2
In thread B, t=3
In thread A, t=2
In thread B, t=4
In thread B, t=5
In thread A, t=3
In thread B, t=6
In thread B, t=7
In thread A, t=4
In thread B, t=8
In thread B, t=9
In thread A, t=5
In thread B, t=10
In thread A, t=6
Thread B is down.
In thread A, t=7
In thread A, t=8
In thread A, t=9
In thread A, t=10
Thread A is down.


下面展示一个实际的例子，我们使用一个列表存储一些数字，然后使用多线程对列表中的数字进行一些复杂的运算（这里简单计算平方，然后用time.sleep()函数等待一点点时间），写法如下：

In [5]:
NumList=[i for i in range(10)]
n=0
class square_thread(threading.Thread):
    def __init__(self, i):
        threading.Thread.__init__(self)
        self.i=i
    def run(self):
        global n
        global NumList
        while n<=9:
            print(f"In thread {self.i}, number={NumList[n]}, result={NumList[n]**2}")
            n+=1
            time.sleep(1)
threads=[]
for i in range(3):
    threads.append(square_thread(i))
    threads[i].start()
for i in range(3):
    threads[i].join()

In thread 0, number=0, result=0
In thread 1, number=1, result=1
In thread 2, number=2, result=4
In thread 0, number=3, result=9
In thread 2, number=4, result=16
In thread 1, number=5, result=25
In thread 0, number=6, result=36
In thread 2, number=7, result=49
In thread 1, number=8, result=64
In thread 0, number=9, result=81


如果将以上代码多运行几遍，就会发现以上代码有很大问题：我们使用了一个全局变量n，然而和有可能存在一种情况即两个线程同时对n进行操作的情况，比如某一次的运行结果如下：
```
In thread 0, number=0, result=0
In thread 1, number=1, result=1
In thread 2, number=2, result=4
In thread 0, number=3, result=9
In thread 1, number=4, result=16
In thread 2, number=5, result=25
In thread 0, number=6, result=36
In thread 1, number=7, result=49In thread 2, number=7, result=49

In thread 0, number=9, result=81
```
导致如上结果的原因是线程1和2同时对n进行了操作：thread1首先取得n=7，然后计算，就在此时thread2也取得n=7，然后计算，然后thread1计算完毕，令n自增1，此时thread0还没有跟进，thread2已经结束，又令n自增1，此时n=9，等到thread0反应过来时，取得的n已经=9，而不是8了，所以导致以上代码计算了两边n=7，但是没有计算n=8。

这就是没有加锁导致的不同线程同时访问相同变量时导致的冲突。为了解决这个问题，我们可以使用一些特殊的数据结构，比如**线程安全**的**队列**（**Queue**）。

# 队列

**队列**（**Queue**）是一种特殊的数据结构，其设计时就是线程安全的，意味着队列可以很放心的用在多线程编程中。

我们可以把队列理解为一个银行排队的过程：不断的有新的元素（人）进入，然后取号，多个不同的柜台按照**先入先出**（**First In First Out**, **FIFO**）的原则，按号码叫好办理业务。这里面，每个柜台可以看成是一个单独的处理任务的线程，队列的设计可以保证两个不同的柜台不会叫到同一个号。当然，元素的进入也可以是多线程的，比如我们可以想象银行有几个不同的门，每个门都有一个取号机，每个取号机也是一个线程，取号机也可以保证每个人拿到的号码是不一样的。

当然，队列也可以不是FIFO的，而是LIFO（Last In Fist Out）的，或者具有优先级的，我们接下来以Queue模块为例介绍FIFO的队列，对于LIFO或者带有优先级的队列，可以使用LifoQueue模块或者PriorityQueue模块。

使用Queue模块只需要从queue模块中导入Queue，然后新建一个Queue的实例，使用put方法加入元素，使用get方法取出元素即可。比如：

In [6]:
from queue import Queue

NumQueue=Queue()
for i in range(10):
    NumQueue.put(i)
while NumQueue.qsize()>0:
    print(f"{NumQueue.get()} is get from NumQueue.")

0 is get from NumQueue.
1 is get from NumQueue.
2 is get from NumQueue.
3 is get from NumQueue.
4 is get from NumQueue.
5 is get from NumQueue.
6 is get from NumQueue.
7 is get from NumQueue.
8 is get from NumQueue.
9 is get from NumQueue.


现在，我们可以将放入元素和取出元素放在两个不同的线程里面，这也是一个简单的**生产者-消费者问题**：生产者将数据放入到队列，然后消费者从队列中取出数据进行处理。

接下来，我们重写以上的平方计算问题：

In [7]:
NumQueue=Queue()

## 生产者
class producer(threading.Thread):
    def __init__(self, begin_num):
        super().__init__()
        self.begin_num=begin_num
    def run(self):
        for i in range(5):
            NumQueue.put(self.begin_num+i)
            time.sleep(0.001)
            
## 消费者
class consumer(threading.Thread):
    def __init__(self, i):
        super().__init__()
        self.i=i
    def run(self):
        while NumQueue.qsize()>0:
            num=NumQueue.get()
            print(f"In thread {self.i}, {num}^2={num**2}")
            time.sleep(1)

## 两个生产者，一个从0开始，一个从10开始：
Producer=[]
for i in range(2):
    Producer.append(producer(10*i))
    Producer[i].start()
## 三个消费者
Consumer=[]
for i in range(3):
    Consumer.append(consumer(i))
    Consumer[i].start()
for i in range(3):
    Consumer[i].join()

In thread 0, 0^2=0In thread 1, 10^2=100

In thread 2, 1^2=1
In thread 0, 11^2=121In thread 2, 2^2=4In thread 1, 12^2=144


In thread 2, 3^2=9
In thread 1, 13^2=169In thread 0, 4^2=16

In thread 0, 14^2=196


如上就解决了线程安全问题。